In [1]:
import pandas as pd
import numpy as np
import os
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
from feature_selection import filter_method as ft

In [2]:
df = pd.read_csv('./final_data.csv')
df.head(5)

,uid,card_sts,is_withdrw,is_transfer,is_deposit,is_purchse,cred_limit,bankacct_bal,is_mob_bank,is_etc,...,delay_bal,loan_amt,loan_bal,overdue_class,overdue_flag,credit_amt,owed_int_in,owed_int_out,guar_amount,credit_level
0,320926196803109398,0,0,0,0,0,10000.0,0.000,0,0,...,0.0,100000.0,90000.00,0,0,300000.0,0.0,0.0,0.000,60
1,320926196803109398,1,0,0,0,0,5000.0,0.000,1,1,...,0.0,100000.0,90000.00,0,0,300000.0,0.0,0.0,0.000,60
2,320926197009035373,1,0,0,0,0,22905.0,88.330,1,1,...,0.0,25000.0,25000.00,0,0,300000.0,0.0,0.0,0.000,70
3,320982198509015737,0,0,0,0,0,22905.0,0.000,0,0,...,0.0,608800.0,297378.48,0,0,620000.0,0.0,0.0,123.746,60
4,320982198509015737,1,0,0,0,0,22905.0,16952.879,1,1,...,0.0,608800.0,297378.48,0,0,620000.0,0.0,0.0,123.746,60


In [3]:
x = df.iloc[:, 0:-1]
x.head(5)

,uid,card_sts,is_withdrw,is_transfer,is_deposit,is_purchse,cred_limit,bankacct_bal,is_mob_bank,is_etc,...,norm_bal,delay_bal,loan_amt,loan_bal,overdue_class,overdue_flag,credit_amt,owed_int_in,owed_int_out,guar_amount
0,320926196803109398,0,0,0,0,0,10000.0,0.000,0,0,...,90000.00,0.0,100000.0,90000.00,0,0,300000.0,0.0,0.0,0.000
1,320926196803109398,1,0,0,0,0,5000.0,0.000,1,1,...,90000.00,0.0,100000.0,90000.00,0,0,300000.0,0.0,0.0,0.000
2,320926197009035373,1,0,0,0,0,22905.0,88.330,1,1,...,35000.00,0.0,25000.0,25000.00,0,0,300000.0,0.0,0.0,0.000
3,320982198509015737,0,0,0,0,0,22905.0,0.000,0,0,...,567766.84,0.0,608800.0,297378.48,0,0,620000.0,0.0,0.0,123.746
4,320982198509015737,1,0,0,0,0,22905.0,16952.879,1,1,...,567766.84,0.0,608800.0,297378.48,0,0,620000.0,0.0,0.0,123.746


In [4]:
y = df.iloc[:, -1]
y.head(5)

0    60
1    60
2    70
3    60
4    60
Name: credit_level, dtype: int64

In [5]:
# filter_method: 剔除常量，因为这些特征对于 y 值没有任何的影响
constant_feature = ft.constant_feature_detect(x, threshold=0.9)
constant_feature

8  variables are found to be almost constant


['is_deposit',
 'bad_bal',
 'due_intr',
 'delay_bal',
 'overdue_class',
 'overdue_flag',
 'owed_int_in',
 'owed_int_out']

In [6]:
x.drop(labels=constant_feature, axis=1, inplace=True)
# uid 非数字，对 star_level 也没有影响，去除
x.drop(labels='uid', axis=1, inplace=True)
x.head(5)

,card_sts,is_withdrw,is_transfer,is_purchse,cred_limit,bankacct_bal,is_mob_bank,is_etc,all_bal,norm_bal,loan_amt,loan_bal,credit_amt,guar_amount
0,0,0,0,0,10000.0,0.000,0,0,90000.00,90000.00,100000.0,90000.00,300000.0,0.000
1,1,0,0,0,5000.0,0.000,1,1,90000.00,90000.00,100000.0,90000.00,300000.0,0.000
2,1,0,0,0,22905.0,88.330,1,1,35000.00,35000.00,25000.0,25000.00,300000.0,0.000
3,0,0,0,0,22905.0,0.000,0,0,567766.84,567766.84,608800.0,297378.48,620000.0,123.746
4,1,0,0,0,22905.0,16952.879,1,1,567766.84,567766.84,608800.0,297378.48,620000.0,123.746


In [7]:
# 计算相关系数，相关系数大于 0.7 的特征可以删除
corr = ft.corr_feature_detect(data=x, threshold=0.7)
for i in corr:
    print(i, '\n')

     feature1     feature2      corr
0  is_withdrw  is_transfer  0.999112
4  is_withdrw   is_purchse  0.987522 

    feature1    feature2      corr
2   norm_bal     all_bal  0.993082
15  norm_bal    loan_bal  0.809665
20  norm_bal  credit_amt  0.764720
25  norm_bal    loan_amt  0.748397 



In [8]:
corr_columns = ['is_withdrw', 'norm_bal']
x.drop(columns=corr_columns, axis=1, inplace=True)

In [9]:
# 计算 VIF 共线性，共线性大于 0.7 的特征删除
vif = ft.calc_VIF(x)
vif.head(30)

,index,VIF
0,card_sts,1.969386
1,is_transfer,45.304401
2,is_purchse,45.286027
3,cred_limit,2.437820
4,bankacct_bal,1.254968
5,is_mob_bank,2.533707
6,is_etc,3.070659
7,all_bal,7.815197
8,loan_amt,23.028272
9,loan_bal,14.743578


In [10]:
# is_transfer、is_purchse、共线性太强，剔除
vif_columns = ['is_transfer', 'is_purchse']
x.drop(columns=vif_columns, axis=1, inplace=True)

In [11]:
# 互信息衡量选择最好的 80% 特征
mi = ft.mutual_info(X=x, y=y, select_k=0.8)
print(mi)

Index(['card_sts', 'cred_limit', 'bankacct_bal', 'all_bal', 'loan_amt',
       'loan_bal', 'credit_amt', 'guar_amount'],
      dtype='object')


In [12]:
new_x = x[['card_sts', 'cred_limit', 'bankacct_bal', 'all_bal', 'loan_amt', 'loan_bal', 'credit_amt', 'guar_amount']]
new_x.head(5)

,card_sts,cred_limit,bankacct_bal,all_bal,loan_amt,loan_bal,credit_amt,guar_amount
0,0,10000.0,0.000,90000.00,100000.0,90000.00,300000.0,0.000
1,1,5000.0,0.000,90000.00,100000.0,90000.00,300000.0,0.000
2,1,22905.0,88.330,35000.00,25000.0,25000.00,300000.0,0.000
3,0,22905.0,0.000,567766.84,608800.0,297378.48,620000.0,123.746
4,1,22905.0,16952.879,567766.84,608800.0,297378.48,620000.0,123.746


In [13]:
ans = pd.concat([new_x, y], axis=1)
ans.to_csv('new_credit_data.csv', header=True, index=False, sep=',')